In [ ]:
# imports 

import numpy as np
import heapq
import math
import scipy as sp
from matplotlib import pyplot
from dataclasses import dataclass

In [ ]:
# Person structure
@dataclass
class Person:
    arrival: float # time of arrival
    onboard: float # time they boarded the elevator
    dest: int      # destination
    stops: int = 0 # number of stops waited (not including their dest)

# Elevator structure

@dataclass
class Elevator:
    # ELEVATOR STATE
    clients: list  # index j is list of Persons that want to go to floor j
    path: list # of (floor number, time of arrival)
    index: int # index into elevators list
    direction: int = 0 # 1 for up, 0 for stationary, -1 for down
    size: int = 0 # amount of people inside
    # FOR PERFORMANCE METRICS
    time_last_floor: float = 0 # time of arrival at last serviced floor
    last_floor: int = 0 # last floor serviced
    distance: int = 0 # total distance traveled
    idle: float = 0 # total time spent idle
    idle_a: float = 0 # time that idle period begins
    avg_occupancy: float = 0 # average occupancy rate, accurate to time_last_floor

# Floor structure

# list of pairs (floor, time of press, direction)
# pair of last times that elevator came to service down/up
# NOTE: all new arrivals are generated at time of elevator arrival
# clients stores people who did not fit inside
# ENSURE clients[j] is sorted by time of arrival

@dataclass
# for the lists below
# 0 is dummy value, 1 is for up, 2/-1 is for down 
class Floor:
    clients: list # list of Persons that want to go up/down
    times: list # times of last service for up/down
    number: int # floor number

# Stores the metrics 
@dataclass
class Metrics:
    avg_wait: float = 0
    wait_n: int = 0 # n people avg wait
    avg_service: float = 0
    service_n: int = 0 # n people avg service
    avg_stops: float = 0
    stops_n: int = 0 # n people avg stops before destination

In [ ]:
# Scheduling policy tips and pointers

# RETINK HOW WE DETERMINE WHERE ELEVATORS ARE AT ANY POINT IN TIME...
# WE DEFINETLY NEED TO CALCULATE IT FROM PREVIOUS, EASY BUT BE AWARE

# Either
# an elvator that has same direction as button
# or a dormant elevator

# What if the optimal elevator is alredy scheduled?
# then we shouldn't change anything

# How do we handle ties?

# cases to keep in mind:
# we can insert a new first state in the path
# or somewhere in the middle for a moving elevator

# Needs to adjust time spent idle
# for dormant elevators

# e_clocks has to be updated if dormant elevator is moved
# or first state in elevator path changes

# FATEST ELEVATOR POLICY 

# if tie current "winner" is kept
# maybe other tiebreaking policies...

# floor as floor number not structure
def fastest_elevator(elevators, time, floor, direction, e_clocks):

    # The algorithm below "streams" in the elevators and keeps a running "best" one
    c_e_arrival_t = time  # the time of arrival to rung floor by fastest elevator
    e_ix = -1             # index of elevator to have path modified
    path_ix = -1          # the index to insert into the path
    change = False        # whether we change any path
    first = False         # augmented first state in path

    for i, elevator in enumerate(elevators):
        
        e_floor = elevator.last_floor # current floor of the elevator (rounded down)
        if elevator.direction == direction: # moving
            
            # current floor of elevator based on time elapsed (1 floor traveled per unit of time)
            if elevator.direction == 2:
                e_floor += np.floor(time - elevator.time_last_floor)
            else:
                e_floor -= np.floor(time - elevator.time_last_floor)

            # check to see if insert is at beginning of path
            if ((direction == 2 and 0 <= floor - e_floor and elevator.path[0][0] > floor) or \
              (direction == 0 and 0 <= e_floor - floor and elevator.path[0][0] < floor)) and  \
              elevator.path[0][1] > c_e_arrival_t:
                    c_e_arrival_t = time + np.abs(e_floor - floor)
                    e_ix = i
                    path_ix = -1
                    change = True
                    first = True

            # check insert middle of path
            for j, (e_floor, e_finish_t) in enumerate(elevator.path):
                if (direction == 2 and 0 <= floor - e_floor) or (direction == 0 and 0 <= e_floor - floor):
                    # quickest elevator is already scheduled
                    if e_floor - floor == 0 and e_finish_t < c_e_arrival_t:
                        change = False
                    # uninitalized or faster to set values
                    elif e_ix == -1 or e_finish_t + np.abs(e_floor - floor) < c_e_arrival_t:
                        c_e_arrival_t = e_finish_t + np.abs(e_floor - floor)
                        e_ix = i
                        path_ix = j
                        change = True
                        first = False
                    else: # elevator is past rung floor
                        break

        elif elevator.direction == 1: # dormant
            # unitialized or closer to set values
            if e_ix == -1 or time + np.abs(e_floor - floor) < c_e_arrival_t:
                c_e_arrival_t = time + np.abs(e_floor - floor)
                e_ix = i
                path_ix = 0
                change = True

    # variables now store "closest" path information 
    # so we augment the relevant path and update clocks
    if change:
        elevators[e_ix].path.insert(path_ix, (floor, c_e_arrival_t)) # augment path
        # dormant elevator to move (update idle time)
        if elevators[e_ix].direction == 1:
            elevators[e_ix].direction = direction
            heapq.heappush(e_clocks, [c_e_arrival_t, e_ix, floor])
            elevators[e_ix].idle += time - elevators[e_ix].idle_a
            elevators[e_ix].idle_a = time
        # need to change e_clocks because earliest state changed
        elif first:
            for i, [_, k] in enumerate(e_clocks):
                if k == e_ix:
                    e_clocks[i][0] = c_e_arrival_t
                    e_clocks[i][2] = floor
                    break

In [ ]:
# policy requires even number of elevators: half only travel up, half only travel down

def uni_direction_elevator(elevators, time, floor, direction, capacity, e_clocks):

    # The algorithm below "streams" in the elevators and keeps a running "best" one
    c_e_arrival_t = time  # the time of arrival to rung floor by fastest elevator
    e_ix = -1             # index of elevator to have path modified
    path_ix = -1          # the index to insert into the path
    change = False        # whether we change any path
    first = False         # augmented first state in path

    for i, elevator in enumerate(elevators):
        
        # only use elevator in appropriate direction
        if (direction == 1 and i % 2 != 0) or (direction == -1 and i % 2 == 0):
            continue

        # if full, skip it
        if elevator.size == capacity:
            continue

        e_floor = elevator.last_floor # current floor of the elevator (rounded down)
        if elevator.direction == direction: # moving
            
            # current floor of elevator based on time elapsed (1 floor traveled per unit of time)
            if elevator.direction == 1:
                e_floor += np.floor(time - elevator.time_last_floor)
            else:
                e_floor -= np.floor(time - elevator.time_last_floor)

            # check to see if insert is at beginning of path
            if ((direction == 1 and 0 <= floor - e_floor and elevator.path[0][0] > floor) or \
              (direction == -1 and 0 <= e_floor - floor and elevator.path[0][0] < floor)) and  \
              elevator.path[0][1] > c_e_arrival_t:
                    c_e_arrival_t = time + np.abs(e_floor - floor)
                    e_ix = i
                    path_ix = -1
                    change = True
                    first = True

            # check insert middle of path
            for j, (e_floor, e_finish_t) in enumerate(elevator.path):
                if (direction == 1 and 0 <= floor - e_floor) or (direction == -1 and 0 <= e_floor - floor):
                    # quickest elevator is already scheduled
                    if e_floor - floor == 0 and e_finish_t < c_e_arrival_t:
                        change = False
                    # uninitalized or faster to set values
                    elif e_ix == -1 or e_finish_t + np.abs(e_floor - floor) < c_e_arrival_t:
                        c_e_arrival_t = e_finish_t + np.abs(e_floor - floor)
                        e_ix = i
                        path_ix = j
                        change = True
                        first = False
                else: # elevator is past rung floor
                    break

        elif elevator.direction == 0: # dormant
            if direction == 1 and i % 2 == 0:
                # unitialized or closer to set values
                if e_ix == -1 or time + np.abs(e_floor - floor) < c_e_arrival_t:
                    c_e_arrival_t = time + np.abs(e_floor - floor)
                    e_ix = i
                    path_ix = 0
                    change = True
            elif direction == -1 and i % 2 != 0:
                # unitialized or closer to set values
                if e_ix == -1 or time + np.abs(e_floor - floor) < c_e_arrival_t:
                    c_e_arrival_t = time + np.abs(e_floor - floor)
                    e_ix = i
                    path_ix = 0
                    change = True

    # variables now store "closest" path information 
    # so we augment the relevant path and update clocks
    if change:
        elevators[e_ix].path.insert(path_ix + 1, (floor, c_e_arrival_t)) # augment path
        # dormant elevator to move (update idle time)
        if elevators[e_ix].direction == 0:
            elevators[e_ix].direction = direction
            heapq.heappush(e_clocks, [c_e_arrival_t, e_ix, floor])
            elevators[e_ix].idle += time - elevators[e_ix].idle_a
        # need to change e_clocks because earliest state changed
        elif first:
            for i, [_, k, __] in enumerate(e_clocks):
                if k == e_ix:
                    e_clocks[i][0] = c_e_arrival_t
                    e_clocks[i][2] = floor
                    break

In [ ]:
# Loading policy tips and pointers


# First come first serve loading
# floor structure
def uni_directional_load(elevator, lambdas, floor, time, metrics, max_capacity, e_clocks, b_clocks):

    rng = np.random.default_rng() # rv generation

    ## CLIENTS INSIDE THE ELEVATOR
    # update occupancy and distance metric
    elevator.avg_occupancy = elevator.avg_occupancy * elevator.time_last_floor / time \
         + (time - elevator.time_last_floor) * elevator.size / (time * max_capacity)
    elevator.distance += np.abs(floor.number - elevator.last_floor)
    
    # Unload people and update average service time and stops
    elevator.size -= len(elevator.clients[floor.number])
    for person in elevator.clients[floor.number]:
        metrics.avg_service = metrics.avg_service + (time - person.onboard - metrics.avg_service) / (metrics.service_n + 1)
        metrics.service_n += 1
        metrics.avg_stops = metrics.avg_stops + (person.stops - metrics.avg_stops) / (metrics.stops_n + 1)
        metrics.stops_n += 1
    elevator.clients[floor.number] = [] # clear people from elevator

    # update people's stops waited
    for _floor in elevator.clients:
        for person in _floor:
            person.stops += 1

    ## LOAD EXISTING CLIENTS INTO ELEVATOR

    # Load people in queue leftover from previous elevator arrival at this floor and update average waiting time
    i = 0 # used to "remove" people from floor queue
    for person in floor.clients[elevator.direction]:
        if elevator.size == max_capacity: # too many people
            break
        metrics.avg_wait = metrics.avg_wait + (time - person.arrival - metrics.avg_wait) / (metrics.wait_n + 1)
        metrics.wait_n += 1
        person.onboard = time
        elevator.clients[person.dest].append(person)
        elevator.size += 1
        i += 1

    # clear people loaded from floor queue using FIFO policy
    floor.clients[elevator.direction] = floor.clients[elevator.direction][i:]

    ## LOAD NEW ARRIVALS RETROSPECTIVELY (IF ANY)

    # Some variable updates
    elevator.last_floor = floor.number
    elevator.time_last_floor = time

    # You arrive "early" before the first button press
    if time - floor.times[elevator.direction] < 0:
        elevator.path = elevator.path[1:]
        # no more places to go
        if len(elevator.path) == 0:
            # send elevator to bottom/top of building
            if elevator.index % 2 == 0 and floor.number != 0:
                elevator.path = [0, time + floor.number]
                heapq.heappush(e_clocks, [elevator.path[0][1], elevator.index, elevator.path[0][0]])
                return
            elif elevator.index % 2 != 0 and floor.number != len(elevator.clients)-1:
                elevator.path = [len(elevator.clients)-1, time + len(elevator.clients)-1 - floor.number]
                heapq.heappush(e_clocks, [elevator.path[0][1], elevator.index, elevator.path[0][0]])
                return
            else:
                elevator.direction = 0
                elevator.idle_a = time
        return

    # "End of the line" down elevator reach 0, up elevator reach top
    if (elevator.direction == -1 and floor.number == 0) or \
        (elevator.direction == 1 and floor.number == len(elevator.clients) - 1):
        elevator.direction = 0 # should have automatically emptied out
        elevator.idle_a = time # start idling
        return

    # make cumulative sum buckets
    temp = np.array(lambdas[floor.number]) # cumuluative lambdas for the floor
    # cumulative lambdas going down (or going up)
    total = sum(lambdas[floor.number][:floor.number]) \
            if elevator.direction == -1 else \
            sum(lambdas[floor.number][floor.number:]) 
    temp = temp / total # probabilities p_ij of going from floor i to j
    # cumulative probabilities for arrivals going to destination floor j
    buckets = np.cumsum(temp[:floor.number]) \
              if elevator.direction == -1 else \
              np.cumsum(temp[floor.number:])    


    # generate arrival times 
    num_arrivals = rng.poisson((time - floor.times[elevator.direction]) * total)
    # if there are no previous people waiting add one; the person who rang
    if len(floor.clients[elevator.direction]) == 0:
        num_arrivals += 1
    arrival_times = rng.uniform(floor.times[elevator.direction], time, num_arrivals)
    arrival_times = sorted(arrival_times) 


    # Retrospective generation 
    for i in range(num_arrivals):
        U = rng.uniform(0,1) # generate probability
        # Find corresponding floor
        p = Person(arrival_times[i], arrival_times[i], 0)
        for ix, val in enumerate(buckets):
            if U <= val:
                p.dest = ix + floor.number \
                         if elevator.direction == 1 else \
                         ix
                # load into elevator if can fit inside
                if elevator.size < max_capacity:
                    elevator.clients[p.dest].append(p)
                    elevator.size += 1
                else: # elevator full
                    floor.clients[elevator.direction].append(p) # clients join queue
                break
    
    # update variables and b_clocks
    floor.times[elevator.direction] = time
    if len(floor.clients[elevator.direction]) > 0: # people still waiting so button gets pressed immediately
        heapq.heappush(b_clocks, (time, floor.number, elevator.direction))
        # infinite loop... it should be past it
    else: # generate next arrival
        heapq.heappush(b_clocks, (time + rng.exponential(1/total), floor.number, elevator.direction))
    
        
    # update elevator path
    new_path = []
    if elevator.direction == 1:
        for ix in range(1, len(elevator.clients) - floor.number):
            if (len(elevator.clients[ix + floor.number])) > 0:
                new_path.append([floor.number + ix, time + ix])
    else:
        for ix in range(floor.number)[::-1]:
            if (len(elevator.clients[ix])) > 0:
                new_path.append([ix, time + floor.number - ix])
    elevator.path = new_path

    # update e_clocks
    heapq.heappush(e_clocks, [elevator.path[0][1], elevator.index, elevator.path[0][0]])

In [ ]:
#################### PARAMETERS ####################

# m         : the number of floors in the building
# n         : the number of elevators 
# lambdas   : m * m array; i,j is the rate of arrivals on floor i who want to go to floor j
# capacity  : max capacity of an elevator
# h         : length of simulation
# SCHEDULER : given list of elevator structures, (time, direction, floor of button press), and elevator clocks
#             augments an elevator structure and e_clocks
# LOADER    : given a floor and elevator stuctures
#             updates floor and elevator structures aftder unload and load

#################### SIMULATION DESIGN ####################

# Elevators have predetermined paths
# On button press, scheduler decides which paths change
# When elevator reaches a floor people get off 
# Arrivals are generated retrospectively with a queue to store excess

# Button timings are events on clock and so are elevator arrivals

#################### NOTES ####################

# Floors are labeled 0 ... m - 1
# Assume time to travel between floors is constant
# We'll assume loading happens in 0 time i.e. instantly

# All 2(m-1) buttons are initialized to begin 
# then buttons are pressed after elevator leaves the floor

#################### DATA STRUCTURES ####################

# down  : tuple where index j is rate of clients of floor j going down
# total : tuple where index j is sum of all rates

# elevators : list of elevator structures
# floors    : list of floor structures

# e_clocks : list of [time, elevator index, floor] that represent the next stop for all active elevators
# b_clocks : list of (time, floor number, direction) that represent floor button presses

#################### METRICS ####################
# average service and wait times 
# average capacity 
# average number of stops before destination
# elevator distance traveled
# elevator time spent idle

#################### Dictionary ####################
# c  |-> closest
# e  |-> elevator
# ix |-> index
# t  |-> time
# b  |-> button

def simple_sim(m: int, n: int, lambdas: [[]], capacity: int, h: int, SCHEDULER, LOADER):

    # Initialization 
    rng       = np.random.default_rng()
    t         = 0  # total elapsed time
    e_clocks  = [] # clocks for elevator arrivals
    b_clocks  = [] # clocks for the button presses
    floors    = list( Floor([[] for x in range(3)], [], i) for i in range(m) )
    elevators = list( Elevator([[] for x in range (m)], [], i) for i in range(n) )
    down      = tuple( sum(lambdas[j][:j]) for j in range(m) )
    total     = tuple( sum(lambdas[j]) for j in range(m) )
    metrics   = Metrics()

    # initialize b_clocks with both up and down presses
    for i in range(1, m-1):
        first = [rng.exponential(1/total[i]), i]
        second = [i]
        if rng.uniform(0, 1) <= down[i] / total[i]:
            first.append(0)
            second.insert(0, first[0] + rng.exponential(1/(total[i] - down[i])))
            second.append(2)
            floors[i].times.append(first[1])
            floors[i].times.append(second[1])
        else:
            first.append(2)
            second.insert(0, first[0] + rng.exponential(1/down[i]))
            second.append(0)
            floors[i].times.append(second[1])
            floors[i].times.append(first[1])
        b_clocks.append(tuple(first))
        b_clocks.append(tuple(second))
    
    # add first and last floor
    b_clocks.append((rng.exponential(1/total[0]), 0, 2))
    b_clocks.append((rng.exponential(1/total[-1]), m - 1, 0))
    heapq.heapify(b_clocks)

    # Finite horizon
    while(t <= h):

        if e_clocks == [] or b_clocks[0][0] <= e_clocks[0][0]: # if no elevator arrival or next event is button arrival
        # SCHEDULER SECTION
            time, floor, direction = heapq.heappop(b_clocks)
            t = time
            SCHEDULER(elevators, time, floor, direction, e_clocks)

        else:
        # LOADING SECTION
            time, e_ix, f_ix = heapq.heappop(e_clocks)
            t = time 
            LOADER(elevators[int(e_ix)], lambdas, floors[int(f_ix)], time, metrics, capacity, e_clocks, b_clocks)
    return metrics

In [ ]:
def genLambdas(m):
    myLambdas = []

    for i in range(m):
        row = []
        for j in range(m):
            if i==j:
                row.append(0.0)
            elif j == 0:
                row.append(0.06)
            else:
                row.append(0.02)
        myLambdas.append(row)
    return myLambdas

In [ ]:
print(simple_sim(5, 1, genLambdas(5), 16, 100, fastest_elevator, simple_load))